In [1]:
import pandas as pd
from pandas import json_normalize
from io import StringIO
from datetime import datetime
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
import requests


# An api key is emailed to you when you sign up to a plan
# Get a free API key at https://api.the-odds-api.com/
API_KEY = '83b174c07fd82ad0a78436d7981be174'

SPORT = 'basketball_nba' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h,spreads,totals' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'american' # decimal | american

DATE_FORMAT = 'iso' # iso | unix

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# First get a list of in-season sports
#   The sport 'key' from the response can be used to get odds in the next request
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

sports_response = requests.get(
    'https://api.the-odds-api.com/v4/sports', 
    params={
        'api_key': API_KEY
    }
)


#if sports_response.status_code != 200:
#    print(f'Failed to get sports: status_code {sports_response.status_code}, response body {sports_response.text}')

#else:
#    print('List of in season sports:', sports_response.json())



# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
# This will deduct from the usage quota
# The usage quota cost = [number of markets specified] x [number of regions specified]
# For examples of usage quota costs, see https://the-odds-api.com/liveapi/guides/v4/#usage-quota-costs
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    #print(odds_json)

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

Number of events: 15
Remaining requests 359
Used requests 141


In [3]:
rows = []

# Process the games data
for game in odds_json:
    home_team = game['home_team']
    away_team = game['away_team']
    
    # Extract the DraftKings bookmaker
    draftkings = next((bk for bk in game['bookmakers'] if bk['key'] == 'draftkings'), None)
    if draftkings:
        # Process the markets
        for market in draftkings['markets']:
            market_type = market['key']
            
            # Iterate through outcomes and create rows for both home and away teams
            for outcome in market['outcomes']:
                team_name = outcome['name']
                price = outcome.get('price')
                point = outcome.get('point', None)  # Not all markets have a point
                
                # We want to add rows for both teams
                rows.append({
                    'game_id': game['id'],
                    'sport_key': game['sport_key'],
                    'sport_title': game['sport_title'],
                    'commence_time': game['commence_time'],
                    'team': team_name,
                    'market_type': market_type,
                    'bookmaker': draftkings['title'],
                    'price': price,
                    'point': point
                })

# Create a DataFrame from the rows
df_draftkings = pd.DataFrame(rows)

df_draftkings['oddstime'] = datetime.now()

df_draftkings

,game_id,sport_key,sport_title,commence_time,team,market_type,bookmaker,price,point,oddstime
0,a46fe84c8e56f3da49d324093bf10907,basketball_nba,NBA,2025-03-10T23:30:00Z,Boston Celtics,h2h,DraftKings,-1600,NaN,2025-03-10 17:38:04.296488
1,a46fe84c8e56f3da49d324093bf10907,basketball_nba,NBA,2025-03-10T23:30:00Z,Utah Jazz,h2h,DraftKings,900,NaN,2025-03-10 17:38:04.296488
2,a46fe84c8e56f3da49d324093bf10907,basketball_nba,NBA,2025-03-10T23:30:00Z,Boston Celtics,spreads,DraftKings,-108,-16.5,2025-03-10 17:38:04.296488
3,a46fe84c8e56f3da49d324093bf10907,basketball_nba,NBA,2025-03-10T23:30:00Z,Utah Jazz,spreads,DraftKings,-112,16.5,2025-03-10 17:38:04.296488
4,a46fe84c8e56f3da49d324093bf10907,basketball_nba,NBA,2025-03-10T23:30:00Z,Over,totals,DraftKings,-108,230.5,2025-03-10 17:38:04.296488
...,...,...,...,...,...,...,...,...,...,...
67,7f0a750037d37aeaa688648cffc4a155,basketball_nba,NBA,2025-03-11T02:40:00Z,Sacramento Kings,h2h,DraftKings,120,NaN,2025-03-10 17:38:04.296488
68,7f0a750037d37aeaa688648cffc4a155,basketball_nba,NBA,2025-03-11T02:40:00Z,New York Knicks,spreads,DraftKings,-112,-2.0,2025-03-10 17:38:04.296488
69,7f0a750037d37aeaa688648cffc4a155,basketball_nba,NBA,2025-03-11T02:40:00Z,Sacramento Kings,spreads,DraftKings,-108,2.0,2025-03-10 17:38:04.296488
70,7f0a750037d37aeaa688648cffc4a155,basketball_nba,NBA,2025-03-11T02:40:00Z,Over,totals,DraftKings,-108,223.0,2025-03-10 17:38:04.296488


In [5]:
df_draftkings.to_gbq(destination_table = 'nba_simulation.dk_odds', project_id = 'sportsbook-models', if_exists = 'replace')

/var/folders/_h/x0tr_2xx4275xs_6khl4zbzh0000gn/T/ipykernel_22693/3215003710.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_draftkings.to_gbq(destination_table = 'nba_simulation.dk_odds', project_id = 'sportsbook-models', if_exists = 'replace')
